In [ ]:

# Importing Libraries
import pandas as pd
import mysql.connector as sql
import streamlit as st
import plotly.express as px
import os
import json
from streamlit_option_menu import option_menu
from PIL import Image
# Setting up page configuration

st.set_page_config(page_title= "Phonepe Pulse Data Visualization ",
                   layout= "wide",
                   initial_sidebar_state= "expanded")

st.sidebar.header(":wave: :violet[**Hello! Welcome to the dashboard**]")
# Creating connection with mysql workbench
mydb = sql.connect(host="localhost",
                   user="root",
                   password="12345678",
                   database= "phonepe",
                   port = "3306"
                  )
mycursor = mydb.cursor(buffered=True)
# Creating option menu in the side bar
with st.sidebar:
    selected = option_menu("Menu", ["Home","Top Charts","Explore","About"], 
                icons=["house","graph-up-arrow", "exclamation-circle"],
                menu_icon= "menu-button-wide",
                default_index=0,
                styles={"nav-link": {"font-size": "20px", "text-align": "left", "margin": "-2px", "--hover-color": "#6F36AD"},
                        "nav-link-selected": {"background-color": "#6F36AD"}})
# MENU 1 - HOME
if selected == "Home":
    st.markdown("# :violet[Data Visualization ]")
    st.markdown("## :violet[A User-Friendly Tool Using Streamlit and Plotly]")
    col1,col2 = st.columns([3,2],gap="medium")
    with col1:
        st.write(" ")
        st.write(" ")
        st.markdown("### :violet[Domain :] Fintech")
        st.markdown("### :violet[Technologies used :]  Python, Pandas, MySQL, mysql-connector-python, Streamlit, and Plotly.")
        st.markdown("### :violet[Overview :] In this streamlit web app you can visualize the phonepe pulse data and gain lot of insights on transactions, number of users, top 10 states and districts based on transactions  and which brand has most number of users and so on. Bar charts and Pie charts are used to get some insights.")
        

# MENU 2 - TOP CHARTS
if selected == "Top Charts":
    st.markdown("## :violet[Top Charts]")
    Type = st.sidebar.selectbox("**Type**", ("Transactions", "Users"))
    colum1,colum2= st.columns([1,1.5],gap="large")
    with colum1:
        Year = st.slider("**Year**", min_value=2018, max_value=2023)
        Quarter = st.slider("Quarter", min_value=1, max_value=4)
    
    with colum2:
        st.info(
                """
                #### From this menu we can get insights like :
                - Overall ranking on a particular Year and Quarter.
                - Top 10 State, District based on Total number of transaction and Total amount spent on phonepe.
                - Top 10 State, District based on Total phonepe users and their app opening frequency.
                - Top 10 mobile brands and its percentage based on the how many people use phonepe.
                """,icon="🔍"
                )
        
# Top Charts - TRANSACTIONS    
    if Type == "Transactions":
        col1,col2 = st.columns([1,1],gap="small")
        
        with col1:
            st.markdown("### :violet[State]")
            mycursor.execute(f"select state, sum(Transaction_count) as Total_Transactions_Count, sum(Transaction_amount) as Total from agg_trans where year = {Year} and quarter = {Quarter} group by state order by Total desc limit 10")
            df = pd.DataFrame(mycursor.fetchall(), columns=['State', 'Transactions_Count','Total_Amount'])
            fig = px.pie(df, values='Total_Amount',
                             names='State',
                             title='Top 10',
                             color_discrete_sequence=px.colors.sequential.Agsunset,
                             hover_data=['Transactions_Count'],
                             labels={'Transactions_Count':'Transactions_Count'})

            fig.update_traces(textposition='inside', textinfo='percent+label')
            st.plotly_chart(fig,use_container_width=True)
            
        with col2:
            st.markdown("### :violet[District]")
            mycursor.execute(f"select district , sum(Count) as Total_Count, sum(Amount) as Total from map_trans where year = {Year} and quarter = {Quarter} group by district order by Total desc limit 10")
            df = pd.DataFrame(mycursor.fetchall(), columns=['District', 'Transactions_Count','Total_Amount'])

            fig = px.pie(df, values='Total_Amount',
                             names='District',
                             title='Top 10',
                             color_discrete_sequence=px.colors.sequential.Agsunset,
                             hover_data=['Transactions_Count'],
                             labels={'Transactions_Count':'Transactions_Count'})

            fig.update_traces(textposition='inside', textinfo='percent+label')
            st.plotly_chart(fig,use_container_width=True)
            
       
            
# Top Charts - USERS          
    if Type == "Users":
        col1,col2,col3 = st.columns([2,2,2],gap="small")
        
        with col1:
            st.markdown("### :violet[Brands]")
            if Year == 2022 and Quarter in [2,3,4]:
                st.markdown("#### Sorry No Data to Display for 2022 Qtr 2,3,4")
            else:
                mycursor.execute(f"select brands, sum(count) as Total_Count, avg(percentage)*100 as Avg_Percentage from agg_user where year = {Year} and quarter = {Quarter} group by brands order by Total_Count asc limit 10")
                df = pd.DataFrame(mycursor.fetchall(), columns=['Brand', 'Total_Users','Avg_Percentage'])
                fig = px.bar(df,
                             title='Top 10',
                             x="Total_Users",
                             y="Brand",
                             orientation='h',
                             color='Avg_Percentage',
                             color_continuous_scale=px.colors.sequential.Agsunset)
                st.plotly_chart(fig,use_container_width=True)   
    
        with col2:
            st.markdown("### :violet[District]")
            mycursor.execute(f"select District, sum(Registered_user) as Total_Users, sum(App_opens) as Total_Appopens from map_user where year = {Year} and quarter = {Quarter} group by district order by Total_Users asc limit 10")
            df = pd.DataFrame(mycursor.fetchall(), columns=['District', 'Total_Users','Total_Appopens'])
            df.Total_Users = df.Total_Users.astype(float)
            fig = px.bar(df,
                         title='Top 10',
                         x="Total_Users",
                         y="District",
                         orientation='h',
                         color='Total_Users',
                         color_continuous_scale=px.colors.sequential.Agsunset)
            st.plotly_chart(fig,use_container_width=True)
              
       
            
        with col3:
            st.markdown("### :violet[State]")
            mycursor.execute(f"select State, sum(Registered_user) as Total_Users, sum(App_opens) as Total_Appopens from map_user where year = {Year} and quarter = {Quarter} group by state order by Total_Users desc limit 10")
            df = pd.DataFrame(mycursor.fetchall(), columns=['State', 'Total_Users','Total_Appopens'])
            fig = px.pie(df, values='Total_Users',
                             names='State',
                             title='Top 10',
                             color_discrete_sequence=px.colors.sequential.Agsunset,
                             hover_data=['Total_Appopens'],
                             labels={'Total_Appopens':'Total_Appopens'})

            fig.update_traces(textposition='inside', textinfo='percent+label')
            st.plotly_chart(fig,use_container_width=True)

#menu 3           
if selected == "Explore":
    st.markdown('<h2 style="color:#b551fc">Explore Data</h2>', unsafe_allow_html=True)
    Category = st.sidebar.selectbox("Category", ("Transactions", "Users"))
    Year = st.slider("Select a Year", min_value=2018, max_value=2023)
    Quarter = st.slider("Select a Quarter for the respective Year", min_value=1, max_value=4)

    # -------------------Exploration for Transactions Data-------------------
    if Category == "Transactions":
        # -------Services for which transactions are done using Phonepe-------
        st.markdown('<h2 style="color:#b551fc">Services for which transactions are done using Phonepe</h2>',
                    unsafe_allow_html=True)
        mycursor.execute(
            f"select Transaction_type, sum(Transaction_count) as Total_Transactions, "
            f"sum(Transaction_amount) as Total_Amount from agg_trans where year= {Year} and quarter = {Quarter} "
            f"group by Transaction_type "
            f"order by Transaction_type")
        df = pd.DataFrame(mycursor.fetchall(), columns=['Transaction_Type', 'Total_Transactions', 'Total_Amount'])

        fig = px.bar(df,
                     title='Services for which Phonepe was used for Transactions',
                     x="Transaction_Type",
                     y="Total_Transactions",
                     color='Total_Amount'
                     )
        st.plotly_chart(fig, use_container_width=True)


        # -------Districts total transactions that are done using Phonepe in a particular State-------
        st.markdown('<h2 style="color:#b551fc">Districts total transactions that are done using Phonepe in a '
                    'particular State</h2>', unsafe_allow_html=True)
        st.markdown('<h3 style="color:#54C8D6">Select a State to continue : </h3>',
                    unsafe_allow_html=True)
        states = st.selectbox("States",
                              ['andaman-&-nicobar-islands', 'andhra-pradesh', 'arunachal-pradesh', 'assam',
                               'bihar', 'chandigarh', 'chhattisgarh', 'dadra-&-nagar-haveli-&-daman-&-diu',
                               'delhi', 'goa', 'gujarat', 'haryana', 'himachal-pradesh', 'jammu-&-kashmir',
                               'jharkhand', 'karnataka', 'kerala', 'ladakh', 'lakshadweep', 'madhya-pradesh',
                               'maharashtra', 'manipur', 'meghalaya', 'mizoram', 'nagaland', 'odisha',
                               'puducherry', 'punjab', 'rajasthan', 'sikkim', 'tamil-nadu', 'telangana',
                               'tripura', 'uttar-pradesh', 'uttarakhand', 'west-bengal'])

        mycursor.execute(
            f"select State, District, sum(count) as Total_Transactions, sum(amount) as Total_Amount "
            f"from map_trans where year = {Year} and quarter = {Quarter} and State = '{states}' "
            f"group by State, District,year,quarter "
            f"order by State,District")

        df = pd.DataFrame(mycursor.fetchall(), columns=['State', 'District', 'Total_Transactions', 'Total_Amount'])
        fig = px.bar(df,
                     title=states,
                     x="District",
                     y="Total_Transactions",
                     color='Total_Amount'
                     )
        st.plotly_chart(fig, use_container_width=True)
        
         # -------State data based on Transactions Count-------
        st.markdown('<h2 style="color:#b551fc">State data based on Transactions Count</h2>', unsafe_allow_html=True)
        mycursor.execute(
            f"select state, sum(count) as Total_Transactions, sum(amount) as Total_Amount from map_trans"
            f" where year = {Year} and quarter = {Quarter} group by state "
            f"order by state")
        df1 = pd.DataFrame(mycursor.fetchall(), columns=['State', 'Total_Transactions', 'Total_Amount'])
        df1["Total_Transactions"] = df1["Total_Transactions"].astype(int)
        df2 = pd.read_csv('States.csv')
        df1["State"] = df2

        fig = px.choropleth(df1,
                            geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
                            featureidkey='properties.ST_NM',
                            locations='State',
                            color='Total_Transactions',
                            title='Transaction Count based Data'
                            )

        fig.update_geos(fitbounds="locations", visible=False)
        st.plotly_chart(fig, use_container_width=True)
        
          # -------State data based on Transactions Amount-------
        st.markdown('<h2 style="color:#b551fc">State data based on Transactions Amount</h2>', unsafe_allow_html=True)
        mycursor.execute(
            f"select state, sum(count) as Total_Transactions, sum(amount) as Total_Amount from map_trans "
            f"where year = {Year} and quarter = {Quarter} group by state "
            f"order by state")
        df1 = pd.DataFrame(mycursor.fetchall(), columns=['State', 'Total_Transactions', 'Total_Amount'])
        df2 = pd.read_csv('States.csv')
        df1["State"] = df2

        fig = px.choropleth(df1,
                            geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
                            featureidkey='properties.ST_NM',
                            locations='State',
                            color='Total_Amount',
                            title='Transaction Amount based Data'
                            )

        fig.update_geos(fitbounds="locations", visible=False)
        st.plotly_chart(fig, use_container_width=True)
        
        
           # -------------------Exploration for User Data------------------
    if Category == "Users":
        # -------State data based on App Opens-------
        st.markdown('<h2 style="color:#b551fc">State data based on App Opens</h2>', unsafe_allow_html=True)
        mycursor.execute(
            f"select state, sum(Registered_user) as Total_Users, sum(App_opens) as Total_Appopens from map_user "
            f"where year = {Year} and quarter = {Quarter} group by state "
            f"order by state")
        df1 = pd.DataFrame(mycursor.fetchall(), columns=['State', 'Total_Users', 'Total_Appopens'])
        df1["Total_Appopens"] = df1["Total_Appopens"].astype(float)
        df2 = pd.read_csv('States.csv')
        df1["State"] = df2

        fig = px.choropleth(df1,
                            geojson="https://gist.githubusercontent.com/jbrobst/56c13bbbf9d97d187fea01ca62ea5112/raw/e388c4cae20aa53cb5090210a42ebb9b765c0a36/india_states.geojson",
                            featureidkey='properties.ST_NM',
                            locations='State',
                            color='Total_Appopens'
                            )

        fig.update_geos(fitbounds="locations", visible=False)
        st.plotly_chart(fig, use_container_width=True)



      

     


        # -------Districts total users that use Phonepe in a particular State-------
        st.markdown('<h2 style="color:#b551fc">Districts total users that use Phonepe in a particular State</h2>',
                    unsafe_allow_html=True)
        st.markdown('<h3 style="color:#54C8D6">Select a State to continue : </h3>',
                    unsafe_allow_html=True)
        states = st.selectbox("States",
                              ['andaman-&-nicobar-islands', 'andhra-pradesh', 'arunachal-pradesh', 'assam',
                               'bihar', 'chandigarh', 'chhattisgarh', 'dadra-&-nagar-haveli-&-daman-&-diu',
                               'delhi', 'goa', 'gujarat', 'haryana', 'himachal-pradesh', 'jammu-&-kashmir',
                               'jharkhand', 'karnataka', 'kerala', 'ladakh', 'lakshadweep', 'madhya-pradesh',
                               'maharashtra', 'manipur', 'meghalaya', 'mizoram', 'nagaland', 'odisha',
                               'puducherry', 'punjab', 'rajasthan', 'sikkim', 'tamil-nadu', 'telangana',
                               'tripura', 'uttar-pradesh', 'uttarakhand', 'west-bengal'])

        mycursor.execute(
            f"select State,District,sum(Registered_user) as Total_Users, sum(App_opens) as Total_App_opens "
            f"from map_user where year = {Year} and quarter = {Quarter} and state = '{states}' "
            f"group by State, District,year,quarter "
            f"order by state,district")

        df = pd.DataFrame(mycursor.fetchall(), columns=['State', 'District', 'Total_Users', 'Total_App_opens'])
        fig = px.bar(df,
                     title=states,
                     x="District",
                     y="Total_Users",
                     )
        st.plotly_chart(fig, use_container_width=True)
            

# MENU 4 - ABOUT
if selected == "About":
    col1,col2 = st.columns([3,3],gap="medium")
    with col1:
        st.write(" ")
        st.write(" ")
        st.markdown("### :violet[About PhonePe Pulse:] ")
        st.write("##### BENGALURU, India, On Sept. 3, 2021 PhonePe, India's leading fintech platform, announced the launch of PhonePe Pulse, India's first interactive website with data, insights and trends on digital payments in the country. The PhonePe Pulse website showcases more than 2000+ Crore transactions by consumers on an interactive map of India. With  over 45% market share, PhonePe's data is representative of the country's digital payment habits.")
        
        st.write("##### The insights on the website and in the report have been drawn from two key sources - the entirety of PhonePe's transaction data combined with merchant and customer interviews. The report is available as a free download on the PhonePe Pulse website and GitHub.")
        
        st.markdown("### :violet[About PhonePe:] ")
        st.write("##### PhonePe is an Indian digital payments and financial services company headquartered in Bengaluru, Karnataka, India. PhonePe was founded in December 2015, by Sameer Nigam, Rahul Chari and Burzin Engineer.PhonePe is India's leading fintech platform with over 300 million registered users. Using PhonePe, users can send and receive money, recharge mobile, DTH, pay at stores, make utility payments, buy gold and make investments. PhonePe forayed into financial services in 2017 with the launch of Gold providing users with a safe and convenient option to buy 24-karat gold securely on its platform. PhonePe has since launched several Mutual Funds and Insurance products like tax-saving funds, liquid funds, international travel insurance and Corona Care, a dedicated insurance product for the COVID-19 pandemic among others. PhonePe also launched its Switch platform in 2018, and today its customers can place orders on over 600 apps directly from within the PhonePe mobile app. PhonePe is accepted at 20+ million merchant outlets across Bharat")
        
        st.write("**:violet[My Project GitHub link]** ")
        st.write("https://github.com/goutham13github/phonepe_pulse")
       
        st.write(" ")
        st.write(" ")
        st.write(" ")
        st.write(" ")
    

